### Proyecto Final Analítica de Datos
Pontificia Universidad Javeriana

Facultad de Ingeniería

Departamento de Ingeniería de Sistemas


Autores: Juan Clavijo, Santiago Camacho, Juan González

Bogotá. Noviembre del 2023

In [ ]:
#Se importan las bibliotecas pertinentes para el desarrollo del proyecto
import pandas as pd
import numpy as np
import time ##Para medir rendimiento de los modelos
import matplotlib.pyplot as plt 
import seaborn as sns
#Se importan las librerías pertinentes de PySpark 
import pyspark as pk
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col 
from pyspark.sql.types import IntegerType, FloatType

In [ ]:
#Se crean dos dataframes, uno para cada set de datos. El primero corresponde al set de datos de areas deforestadas en el depaertamento del chocó en Colombia. El segundo corresponde a los datos de nacimiento de algunos municipios en Colombia. 
areas_deforestadas=pd.read_csv("https://raw.githubusercontent.com/jclavijomartinez/analiticaDeDatos2330/master/datos%20proyecto%20final/AREAS_DEFORESTADAS_CHOCO.csv")
# Para el dataframe de nacimientos se carga el archivo CSV especificando el delimitador como punto y coma ya que en el archivo CSV no se utiliza la coma como delimitador sino el ;
nacimientos = pd.read_csv("https://raw.githubusercontent.com/jclavijomartinez/analiticaDeDatos2330/master/datos%20proyecto%20final/finaldatosnacimientos.csv", sep=';')


In [ ]:
#Se imprime la información del dataframe de areas deforestadas del chocó.
areas_deforestadas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7937 entries, 0 to 7936
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              7937 non-null   int64  
 1   TIPO GEOMETRIA  7937 non-null   object 
 2   AÑO             7937 non-null   int64  
 3   IMAGEN          5901 non-null   object 
 4   CAUSA           7718 non-null   object 
 5   AREA_Ha         7937 non-null   float64
 6   OBSERVACION     7937 non-null   object 
 7   LATITUD         7930 non-null   object 
 8   LONGITUD        7930 non-null   object 
 9   MUNICIPIO       7937 non-null   object 
dtypes: float64(1), int64(2), object(7)
memory usage: 620.2+ KB


In [ ]:
#Se imprime la información del dataframde de nacimientos en los municipios del país.
nacimientos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 955 entries, 0 to 954
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   año                       955 non-null    int64  
 1   departamento              955 non-null    object 
 2   municipio                 955 non-null    object 
 3   total                     955 non-null    float64
 4   total H                   955 non-null    object 
 5   total M                   955 non-null    float64
 6   total indet               955 non-null    object 
 7   cabecera municipal H      955 non-null    object 
 8   cabecera municipal M      955 non-null    float64
 9   cebecera municipal indet  955 non-null    object 
 10  centro poblado H          955 non-null    float64
 11  centro poblado M          955 non-null    float64
 12  centro poblado indet      955 non-null    object 
 13  rural disperso H          955 non-null    float64
 14  rural disp

### Se evidencia que en ambos dataframes los valores de las columnas no son los mismos, lo que indica que hay datos duplicados y/o faltantes. A continuación se realizará la limpieza de los datasets teniendo como objetivo eliminar los dataos faltantes y los datos duplicados. 

In [ ]:
##Cuenta de datas null, etc... para el dataframe de nacimientos     
desaparecidos = len(nacimientos)-len(nacimientos.dropna())

print('Cantidad de observaciones con Datos NaN', desaparecidos)
print('Cantidad de datos duplicados', nacimientos.duplicated().sum())

Cantidad de observaciones con Datos NaN 192
Cantidad de datos duplicados 0


In [ ]:
##Cuenta de datas null, etc... para el dataframe de areas deforestadas
desaparecidos = len(areas_deforestadas)-len(areas_deforestadas.dropna())

print('Cantidad de observaciones con Datos NaN', desaparecidos)
print('Cantidad de datos duplicados', areas_deforestadas.duplicated().sum())

Cantidad de observaciones con Datos NaN 2262
Cantidad de datos duplicados 0


In [ ]:
## Se eliminan los datos Null y Duplicados para ambos dataframes
areas_deforestadas.dropna(inplace=True)
areas_deforestadas.drop_duplicates(inplace = True)
nacimientos.dropna(inplace=True)
nacimientos.drop_duplicates(inplace=True)
#Se reinicia el indice por los eliminados (duplicados)
areas_deforestadas.reset_index(drop = True, inplace = True)
nacimientos.reset_index(drop = True, inplace = True)

In [ ]:
#Se imprime la nueva información del dataset de nacimientos por municipio
nacimientos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 763 entries, 0 to 762
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   año                       763 non-null    int64  
 1   departamento              763 non-null    object 
 2   municipio                 763 non-null    object 
 3   total                     763 non-null    float64
 4   total H                   763 non-null    object 
 5   total M                   763 non-null    float64
 6   total indet               763 non-null    object 
 7   cabecera municipal H      763 non-null    object 
 8   cabecera municipal M      763 non-null    float64
 9   cebecera municipal indet  763 non-null    object 
 10  centro poblado H          763 non-null    float64
 11  centro poblado M          763 non-null    float64
 12  centro poblado indet      763 non-null    object 
 13  rural disperso H          763 non-null    float64
 14  rural disp

In [ ]:
#Se imprime la nueva información del dataset de areas deforestadas
areas_deforestadas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5675 entries, 0 to 5674
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              5675 non-null   int64  
 1   TIPO GEOMETRIA  5675 non-null   object 
 2   AÑO             5675 non-null   int64  
 3   IMAGEN          5675 non-null   object 
 4   CAUSA           5675 non-null   object 
 5   AREA_Ha         5675 non-null   float64
 6   OBSERVACION     5675 non-null   object 
 7   LATITUD         5675 non-null   object 
 8   LONGITUD        5675 non-null   object 
 9   MUNICIPIO       5675 non-null   object 
dtypes: float64(1), int64(2), object(7)
memory usage: 443.5+ KB


### Se evidencia que se eliminaron los registros duplicados y valores faltantes para ambos datasets. Es decir, todas las columnas de ambos datasets tienen exactamente el mismo número de registros. 

### A continuación se cambiarán los nombres de algunas columnas de los datasets para que cumplan con el estándar PEP8

In [ ]:
#Se cambian los nombres de ciertas columnas para que cumplan el estándar PEP8
areas_deforestadas = areas_deforestadas.rename(columns={'TIPO GEOMETRIA': 'TIPO_GEOMETRIA'})
nacimientos = nacimientos.rename(columns={'total H': 'total_H'})
nacimientos = nacimientos.rename(columns={'total M': 'total_M'})
nacimientos = nacimientos.rename(columns={'total indet': 'total_indet'})
nacimientos = nacimientos.rename(columns={'cabecera municipal H': 'cabecera_municipal_H'})
nacimientos = nacimientos.rename(columns={'cabecera municipal M': 'cabecera_municipal_M'})
nacimientos = nacimientos.rename(columns={'cebecera municipal indet': 'cabecera_municipal_indet'})
nacimientos = nacimientos.rename(columns={'centro poblado H': 'centro_poblado_H'})
nacimientos = nacimientos.rename(columns={'centro poblado M': 'centro_poblado_M'})
nacimientos = nacimientos.rename(columns={'centro poblado indet': 'centro_poblado_indet'})
nacimientos = nacimientos.rename(columns={'rural disperso H': 'rural_disperso_H'})
nacimientos = nacimientos.rename(columns={'rural disperso M': 'rural_disperso_M'})
nacimientos = nacimientos.rename(columns={'rural disperso indet': 'rural_disperso_indet'})
nacimientos = nacimientos.rename(columns={'sin info H': 'sin_info_H'})
nacimientos = nacimientos.rename(columns={'sin info M': 'sin_info_M'})
nacimientos = nacimientos.rename(columns={'sin info indet': 'sin_info_indet'})




In [ ]:
#Se imprime la nueva información del dataset de areas deforestadas
areas_deforestadas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5675 entries, 0 to 5674
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              5675 non-null   int64  
 1   TIPO_GEOMETRIA  5675 non-null   object 
 2   AÑO             5675 non-null   int64  
 3   IMAGEN          5675 non-null   object 
 4   CAUSA           5675 non-null   object 
 5   AREA_Ha         5675 non-null   float64
 6   OBSERVACION     5675 non-null   object 
 7   LATITUD         5675 non-null   object 
 8   LONGITUD        5675 non-null   object 
 9   MUNICIPIO       5675 non-null   object 
dtypes: float64(1), int64(2), object(7)
memory usage: 443.5+ KB


In [ ]:
#Se imprime la nueva información del dataset de nacimientos
nacimientos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 763 entries, 0 to 762
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   año                       763 non-null    int64  
 1   departamento              763 non-null    object 
 2   municipio                 763 non-null    object 
 3   total                     763 non-null    float64
 4   total_H                   763 non-null    object 
 5   total_M                   763 non-null    float64
 6   total_indet               763 non-null    object 
 7   cabecera_municipal_H      763 non-null    object 
 8   cabecera_municipal_M      763 non-null    float64
 9   cabecera_municipal_indet  763 non-null    object 
 10  centro_poblado_H          763 non-null    float64
 11  centro_poblado_M          763 non-null    float64
 12  centro_poblado_indet      763 non-null    object 
 13  rural_disperso_H          763 non-null    float64
 14  rural_disp

Se imprimen los primeros 15 registros de cada dataframe

In [ ]:
nacimientos.head(15)

,año,departamento,municipio,total,total_H,total_M,total_indet,cabecera_municipal_H,cabecera_municipal_M,cabecera_municipal_indet,centro_poblado_H,centro_poblado_M,centro_poblado_indet,rural_disperso_H,rural_disperso_M,rural_disperso_indet,sin_info_H,sin_info_M,sin_info_indet
0,2009,antioquia,total,85.955,44.173,41.782,"0,00",33.222,31.279,"0,00",3.249,3.032,"0,00",6.393,6.137,"0,00",1.309,1.334,"0,00"
1,2009,antioquia,MEDELLÍN,31.893,16.334,15.559,"0,00",15.871,15.141,"0,00",210.000,208.000,"0,00",126.000,101.000,"0,00",127.000,109.000,"0,00"
2,2009,antioquia,ABEJORRAL,271.000,137,134.000,"0,00",50,43.000,"0,00",9.000,5.000,"0,00",67.000,74.000,"0,00",11.000,12.000,"0,00"
3,2009,antioquia,ABRIAQUÍ,21.000,11,10.000,"0,00",4,6.000,"0,00",1.000,1.000,"0,00",6.000,3.000,"0,00",0.000,0.000,"0,00"
4,2009,antioquia,ALEJANDRÍA,44.000,17,27.000,"0,00",4,14.000,"0,00",0.000,0.000,"0,00",13.000,11.000,"0,00",0.000,2.000,"0,00"
5,2009,antioquia,AMAGÁ,354.000,173,181.000,"0,00",73,97.000,"0,00",37.000,33.000,"0,00",55.000,42.000,"0,00",8.000,9.000,"0,00"
6,2009,antioquia,AMALFI,404.000,205,199.000,"0,00",104,83.000,"0,00",7.000,7.000,"0,00",92.000,107.000,"0,00",2.000,2.000,"0,00"
7,2009,antioquia,ANDES,594.000,301,293.000,"0,00","3,00",117.000,"0,00",82.000,58.000,"0,00",83.000,102.000,"0,00",15.000,16.000,"0,00"
8,2009,antioquia,ÁNGELÓPOLIS,73.000,39,34.000,"0,00",15,14.000,"0,00",8.000,10.000,"0,00",15.000,8.000,"0,00",1.000,2.000,"0,00"
9,2009,antioquia,ANGOSTURA,160.000,90,70.000,"0,00",14,14.000,"0,00",0.000,1.000,"0,00",70.000,51.000,"0,00",6.000,4.000,"0,00"


In [ ]:
areas_deforestadas.head(15)

,ID,TIPO_GEOMETRIA,AÑO,IMAGEN,CAUSA,AREA_Ha,OBSERVACION,LATITUD,LONGITUD,MUNICIPIO
0,7463,Polygon,2020,T18PTQ_20200201T153611_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.126512,Ninguna,"8° 22' 43,726"" N","77° 14' 19,019"" W",ACANDÍ
1,4659,Polygon,2019,T18PTQ_20190527T153621_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.140720,Ninguna,"8° 30' 47,551"" N","77° 19' 30,507"" W",ACANDÍ
2,7459,Polygon,2020,T18PTQ_20200112T153611_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.141250,Ninguna,"8° 22' 30,966"" N","77° 13' 49,720"" W",ACANDÍ
3,9719,Polygon,2021,T18PTQ_20201202T153619_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.158066,Ninguna,"8° 36' 11.256"" N","77° 20' 50.757"" W",ACANDÍ
4,7464,Polygon,2020,T18PTQ_20200201T153611_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.184549,Ninguna,"8° 22' 22,008"" N","77° 14' 3,921"" W",ACANDÍ
5,7445,Polygon,2020,T18PTQ_20200112T153611_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.184854,Ninguna,"8° 20' 7,286"" N","77° 12' 42,121"" W",ACANDÍ
6,7440,Polygon,2020,T18PTQ_20201107T153621_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.185392,Ninguna,"8° 23' 18,620"" N","77° 14' 40,196"" W",ACANDÍ
7,7484,Polygon,2020,T18PTQ_20200211T153611_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.200436,Ninguna,"8° 27' 50,102"" N","77° 21' 38,627"" W",ACANDÍ
8,7478,Polygon,2020,T18PTQ_20200211T153611_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.214739,Ninguna,"8° 24' 2,397"" N","77° 9' 34,027"" W",ACANDÍ
9,4684,Polygon,2019,T18PTQ_20190527T153621_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.217124,Ninguna,"8° 29' 1,740"" N","77° 22' 8,066"" W",ACANDÍ
